In [2]:
# Programming evacuation systems using Crowd Simulation

# Loading the pygame package
import pygame
import import_ipynb

# Importing locals
from pygame.locals import *
from additional_functions import *
from environment_2F import *
from agent import *

# Other packages
import sys
import numpy as np
import numpy.random as random
import math
import time


# data_matrix = np.loadtxt('room1_r', dtype=float) # Enable after first run
data_matrix = np.zeros((nr_experiments * nr_agents, 4))  # Delete/comment after first run

# Making sure we can run experiments one by one
j = 0  # add one after running

# Checking if all are executed
if j == nr_agents:
    print("nr of experiments reached")
    sys.exit()

# Initializing Pygame and font
pygame.init()
pygame.font.init()
timefont = pygame.font.SysFont('Comic Sans MS', 30)

""" 

Creating a screen with a room that is smaller than then screen 

"""

# Size of the screen
width = 1500
height = 800
size = width, height  # Do not adjust this

# Creating screen
roomscreen = pygame.display.set_mode(size)

# Making background white and creating colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
background_color = WHITE
roomscreen.fill(background_color)
pygame.display.update()

# Defining clock
clock = pygame.time.Clock()

# Move world view
world_offset = [0,0]

def main():
    # Now to let multiple objects move to the door we define
    # nr_agents = nr_agents
    agent_color = GREEN
    line_color = BLACK

    # initialize agents
    agents = []

    def positions(agents):
        for i in range(nr_agents):
            agent = Agent()
            agent.walls = walls
            agent.x = positionmatrix[j * nr_agents + i][0]
            agent.y = positionmatrix[j * nr_agents + i][1]
            agent.pos = np.array([agent.x, agent.y])
            agent.radius = positionmatrix[j * nr_agents + i][2]
            agent.mass = positionmatrix[j * nr_agents + i][3]
            agent.dSpeed = positionmatrix[j * nr_agents + i][4]
            agents.append(agent)

    positions(agents)

    count = 0
    start_time = time.time()
    run = True

    while run:

        # Updating time
        if count < nr_agents:
            current_time = time.time()
            elapsed_time = current_time - start_time
        else:
            for agent_i in agents:
                data_matrix[(j + 1) * nr_agents - 2][0] = elapsed_time
                data_matrix[(j + 1) * nr_agents - 1][0] = elapsed_time
                agents.remove(agent_i)
            for k in range(j * nr_agents, (j + 1) * nr_agents):
                data_matrix[k][1] = elapsed_time

        # Finding delta t for this frame
        dt = clock.tick(70) / 1000

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                (x, y) = pygame.mouse.get_pos()
                print(x, y)

        global world_offset

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            world_offset[0] = world_offset[0] + 30
        if keys[pygame.K_RIGHT]:
            world_offset[0] = world_offset[0] - 30
        if keys[pygame.K_UP]:
            world_offset[1] = world_offset[1] + 30
        if keys[pygame.K_DOWN]:
            world_offset[1] = world_offset[1] - 30

        roomscreen.fill(background_color)

        # draw walls
        for wall in walls:
            start_posw = np.array([wall[0], wall[1]])
            end_posw = np.array([wall[2], wall[3]])
            start_posx = start_posw
            end_posx = end_posw
            pygame.draw.line(roomscreen, line_color, start_posx + world_offset, end_posx + world_offset, 3)

        for agent_i in agents:
            agent_i.update_dest()
            agent_i.direction = normalize(agent_i.dest - agent_i.pos)
            agent_i.dVelocity = agent_i.dSpeed * agent_i.direction
            aVelocity_force = agent_i.velocity_force()
            people_interaction = 0.0
            wall_interaction = 0.0

            for agent_j in agents:
                if agent_i == agent_j: continue
                people_interaction += agent_i.f_ij(agent_j)

            for wall in walls:
                wall_interaction += agent_i.f_ik_wall(wall)

            sumForce = aVelocity_force + people_interaction + wall_interaction
            dv_dt = sumForce / agent_i.mass
            agent_i.aVelocity = agent_i.aVelocity + dv_dt * dt
            agent_i.pos = agent_i.pos + agent_i.aVelocity * dt

            # Avoiding disappearing agents
            if agent_i.pos[0] > 2900 or agent_i.pos[0] < 50 or agent_i.pos[1] > 2300 or agent_i.pos[1] < 50:
                main()
                sys.exit()

        for agent_i in agents:

            agent_i.time += clock.get_time() / 1000
            start_position = [0, 0]
            start_position[0] = int(agent_i.pos[0]) + world_offset[0]
            start_position[1] = int(agent_i.pos[1]) + world_offset[1]

            end_position = [0, 0]
            end_position[0] = int(agent_i.pos[0] + agent_i.aVelocity[0]) + world_offset[0]
            end_position[1] = int(agent_i.pos[1] + agent_i.aVelocity[1]) + world_offset[1]

            end_positionDV = [0, 0]
            end_positionDV[0] = int(agent_i.pos[0] + agent_i.dVelocity[0]) + world_offset[0]
            end_positionDV[1] = int(agent_i.pos[1] + agent_i.dVelocity[1]) + world_offset[1]

            if ((start_position[0] >= 1156.3/2 + 4203/2 + buffer + world_offset[0]) or (start_position[1] >= 751.1/2 + (3 * 885.8/2) + 321.6/2 + buffer + world_offset[1] and start_position[0] <= buffer + 735/2 + 324/2 + world_offset[0]) ) and agent_i.Goal == 0:
                agent_i.Goal = 1
                data_matrix[count + j * nr_agents][0] = agent_i.time
                # print('Time to Reach the Goal:', agent_i.time)

            if (start_position[0] >= 1156.3/2 + 4203/2 + buffer + world_offset[0]) or (start_position[1] >= 751.1/2 + (3 * 885.8/2) + 321.6/2 + buffer + world_offset[1] and start_position[0] <= buffer + 735/2 + 324/2 + world_offset[0]):
                data_matrix[count + j * nr_agents][2] = count
                data_matrix[count + j * nr_agents][3] = agent_i.countcollision
                count += 1
                agents.remove(agent_i)

            pygame.draw.circle(roomscreen, agent_color, start_position, round(agent_i.radius), 3)
            pygame.draw.line(roomscreen, agent_color, start_position, end_positionDV, 2)
        # visibility
        # pygame.draw.circle(roomscreen, RED, [700,400], 300, 1)

        pygame.draw.line(roomscreen, [255, 60, 0], start_position, end_positionDV, 2)

        # Present text on screen
        timestr = "Time: " + str(elapsed_time)
        agentstr = "Number of Agents Escaped: " + str(count) + "/" + str(nr_agents)
        timesurface = timefont.render(timestr, False, (0, 0, 0))
        agentsurface = timefont.render(agentstr, False, (0, 0, 0))
        roomscreen.blit(timesurface, (0, 0))
        roomscreen.blit(agentsurface, (0, 40))
        # Update the screen
        pygame.display.flip()

    pygame.quit()


main()
# np.savetxt('room1_vis', data_matrix)


X and Y Position: [437.49183426 495.13797101]
self.direction: [0.34379968 0.93904301]
X and Y Position: [ 201.47109048 1849.93199094]
self.direction: [ 0.96007282 -0.27975021]
X and Y Position: [ 500.15462272 1207.52977521]
self.direction: [-0.85824351  0.51324271]
X and Y Position: [ 220.4821696 1842.7058524]
self.direction: [ 0.15906963 -0.98726737]
X and Y Position: [ 951.66810158 1784.83286608]
self.direction: [-0.81544054 -0.57884085]
X and Y Position: [1147.61014746 1839.75319431]
self.direction: [-0.70398631 -0.71021354]
X and Y Position: [ 235.59664288 1154.67161471]
self.direction: [0.86166934 0.50747014]
X and Y Position: [660.96395614 632.88657631]
self.direction: [0.9857517 0.168207 ]
X and Y Position: [330.8151338  241.05259188]
self.direction: [0.13008938 0.99150227]
X and Y Position: [1042.00273512  727.97692432]
self.direction: [-0.1592775   0.98723385]
X and Y Position: [157.53444858 246.95262818]
self.direction: [0.0406642  0.99917287]
X and Y Position: [ 224.21777713

X and Y Position: [437.31034641 565.90941689]
self.direction: [-0.00209874  0.9999978 ]
X and Y Position: [ 501.13556803 1162.67286932]
self.direction: [-0.67901888  0.73412081]
X and Y Position: [861.73901501 759.92113958]
self.direction: [-0.3080136   0.95138195]
X and Y Position: [ 425.58528588 1826.39387078]
self.direction: [-0.83248961 -0.55404066]
X and Y Position: [ 266.7178716  1172.74347287]
self.direction: [0.76440496 0.64473642]
X and Y Position: [902.65427671 655.88301022]
self.direction: [0.00164131 0.99999865]
X and Y Position: [ 126.14340495 1841.69691839]
self.direction: [ 0.3762557  -0.92651587]
X and Y Position: [1059.98560064  688.60058107]
self.direction: [-0.98235371  0.18703259]
X and Y Position: [ 198.96568275 1125.05405904]
self.direction: [0.94823523 0.31756881]
X and Y Position: [773.13008537 133.53852224]
self.direction: [-0.33042854  0.94383101]
X and Y Position: [810.04781537 323.58269804]
self.direction: [-0.23098159  0.97295812]
X and Y Position: [ 239.12

X and Y Position: [ 727.55885723 1334.01649335]
self.direction: [-0.74903178 -0.66253407]
X and Y Position: [1117.48006564 1190.7876838 ]
self.direction: [-0.83966417  0.54310595]
X and Y Position: [956.18630984 130.49063463]
self.direction: [0.11171504 0.99374028]
X and Y Position: [ 984.86768528 1682.53329812]
self.direction: [-0.53099339 -0.84737596]
X and Y Position: [642.83613639 557.976752  ]
self.direction: [-0.5284416   0.84896965]
X and Y Position: [ 447.08834715 1859.36619124]
self.direction: [-0.53505386 -0.84481795]
X and Y Position: [1104.51930412 1256.66239212]
self.direction: [-0.99331589 -0.11542763]
X and Y Position: [788.4068824 956.8636098]
self.direction: [0.94912791 0.3148908 ]
X and Y Position: [ 579.23467951 1417.21790788]
self.direction: [-0.57943292 -0.81501993]
X and Y Position: [ 749.25486116 1538.25866485]
self.direction: [ 0.51089131 -0.85964531]
X and Y Position: [ 670.5345614  1814.57848191]
self.direction: [0.99653987 0.08311615]
X and Y Position: [1138.

X and Y Position: [636.0516891  443.70645275]
self.direction: [0.46324133 0.88623217]
X and Y Position: [298.0975553  136.10630451]
self.direction: [0.2669019 0.9637237]
X and Y Position: [385.08891242 469.35194587]
self.direction: [0.79653486 0.60459261]
X and Y Position: [ 709.39265495 1645.69107412]
self.direction: [ 0.29535387 -0.95538793]
X and Y Position: [1018.82243495 1846.42324848]
self.direction: [-0.92304822 -0.38468427]
X and Y Position: [ 909.84954327 1120.60093002]
self.direction: [ 0.39854708 -0.91714788]
X and Y Position: [848.39180158 888.94342044]
self.direction: [0.99540684 0.09573514]
X and Y Position: [381.35436824 567.34793208]
self.direction: [-0.86866751  0.49539556]
X and Y Position: [800.66031539 609.00569491]
self.direction: [0.49079335 0.87127601]
X and Y Position: [556.98097237 472.85030178]
self.direction: [0.89536158 0.44533992]
X and Y Position: [264.52058507 691.88857047]
self.direction: [0.25334425 0.96737619]
X and Y Position: [ 255.77615308 1174.2211

X and Y Position: [558.46186841 522.11867756]
self.direction: [0.14479155 0.98946218]
X and Y Position: [ 825.78601561 1553.0646504 ]
self.direction: [-0.92017964 -0.39149639]
X and Y Position: [ 393.68125135 1744.56997484]
self.direction: [ 0.59115687 -0.80655661]
X and Y Position: [277.84630397 438.38640418]
self.direction: [-0.2017644   0.97943409]
X and Y Position: [ 170.5254056  1566.25008332]
self.direction: [ 0.85441166 -0.51959669]
X and Y Position: [ 398.46797461 1199.14977267]
self.direction: [ 0.697015   -0.71705655]
X and Y Position: [ 799.16336843 1142.677122  ]
self.direction: [-0.92718035  0.37461528]
X and Y Position: [ 517.26046673 1516.83717835]
self.direction: [-0.16530012 -0.98624331]
X and Y Position: [176.9960197  486.72645623]
self.direction: [0.83480453 0.55054645]
X and Y Position: [988.46855394 208.2947155 ]
self.direction: [-0.61410964  0.78922073]
X and Y Position: [1128.25272054  771.27637155]
self.direction: [-0.91635921  0.40035709]
X and Y Position: [111

...

X and Y Position: [1174.7772118  1606.57433318]
self.direction: [-0.34665834 -0.93799147]
X and Y Position: [925.05743413 248.62278942]
self.direction: [-0.05562806  0.99845156]
X and Y Position: [ 171.88048584 1291.97928855]
self.direction: [0.99113457 0.1328618 ]
X and Y Position: [ 482.5839556  1240.33635353]
self.direction: [-0.99997296 -0.00735443]
X and Y Position: [ 916.04965141 1812.86687162]
self.direction: [ 0.32635522 -0.9452472 ]
X and Y Position: [ 470.3413742  1065.26369596]
self.direction: [-0.60292477  0.79779805]
X and Y Position: [ 334.79879142 1332.18841304]
self.direction: [0.96729539 0.25365256]
X and Y Position: [201.71420765 336.69561868]
self.direction: [0.72680919 0.68683943]
X and Y Position: [841.5316297  442.86245518]
self.direction: [-0.33439927  0.9424315 ]
X and Y Position: [581.76261157 505.98177599]
self.direction: [0.35465076 0.93499884]
X and Y Position: [ 366.35528568 1056.5511197 ]
self.direction: [0.98446978 0.17555414]
X and Y Position: [1142.9812

X and Y Position: [1103.41115835  680.60474326]
self.direction: [0.13991637 0.99016332]
X and Y Position: [522.59837716 479.57625657]
self.direction: [0.49299471 0.87003231]
X and Y Position: [ 630.81852837 1169.51320399]
self.direction: [0.55410866 0.83244435]
X and Y Position: [ 863.92099354 1162.67145769]
self.direction: [-0.78884606  0.61459083]
X and Y Position: [1132.7703338  1787.62042722]
self.direction: [-0.73606983 -0.67690561]
X and Y Position: [593.13020276 878.53673472]
self.direction: [0.58863394 0.80839971]
X and Y Position: [786.50124936 171.93207451]
self.direction: [0.11011301 0.99391907]
X and Y Position: [189.20081454 382.17911727]
self.direction: [0.33077265 0.94371047]
X and Y Position: [1108.55926983  396.71396299]
self.direction: [-0.17299469  0.98492276]
X and Y Position: [ 332.2199367  1421.82272973]
self.direction: [0.69398695 0.71998758]
X and Y Position: [507.50758364 406.79174166]
self.direction: [0.53174032 0.84690745]
X and Y Position: [185.60065179 720.

X and Y Position: [ 660.93238229 1429.23909113]
self.direction: [ 0.96921609 -0.24621164]
X and Y Position: [ 155.98470574 1475.27132933]
self.direction: [ 0.44168534 -0.89717003]
X and Y Position: [ 140.1928696  1116.82725435]
self.direction: [0.85929786 0.5114755 ]
X and Y Position: [134.57672543 662.93410879]
self.direction: [0.98469071 0.17431065]
X and Y Position: [543.01350295 508.48836606]
self.direction: [0.14354345 0.98964402]
X and Y Position: [701.68596688 960.00054677]
self.direction: [0.22513707 0.9743271 ]
X and Y Position: [ 312.54697148 1621.6386587 ]
self.direction: [ 0.46458451 -0.88552879]
X and Y Position: [ 493.37336676 1458.39682633]
self.direction: [ 0.07609899 -0.99710027]
X and Y Position: [611.87051558 298.17094256]
self.direction: [0.37300647 0.92782874]
X and Y Position: [ 129.78629199 1869.5275397 ]
self.direction: [ 0.09088983 -0.99586095]
X and Y Position: [ 451.87101894 1512.47758426]
self.direction: [ 0.80772537 -0.58955893]
X and Y Position: [ 294.5865

X and Y Position: [ 655.50084059 1621.40387133]
self.direction: [ 0.47003054 -0.88265015]
X and Y Position: [528.97255177 555.802974  ]
self.direction: [0.92856591 0.37116756]
X and Y Position: [1070.04224822  608.57694549]
self.direction: [-0.61560069  0.78805824]
X and Y Position: [ 900.0499912  1257.21361082]
self.direction: [-0.90773102 -0.4195526 ]
X and Y Position: [475.90982693 289.05939237]
self.direction: [-0.48097407  0.87673482]
X and Y Position: [ 414.60478155 1468.07560458]
self.direction: [ 0.99040653 -0.13818431]
X and Y Position: [ 679.06659368 1240.24239033]
self.direction: [ 0.59738293 -0.80195613]
X and Y Position: [921.42121191 763.73563171]
self.direction: [0.31615082 0.94870894]
X and Y Position: [1094.7247334  864.6721191]
self.direction: [-0.59239308  0.80564908]
X and Y Position: [773.92367133 933.06236935]
self.direction: [0.47894004 0.87784762]
X and Y Position: [282.88742067 903.78507298]
self.direction: [0.9264092  0.37651826]
X and Y Position: [ 175.935868

SystemExit: 